
  # Main code

In [3]:
using BSeries
using SymPy
using LinearAlgebra
using PyCall

# This function generates a polinomial 
#       A_{t,z} = [t,t^2/2,..., t^s/s]*M*[1, z, ..., z^(s-1)]^T
# for a given square matrix M of dimmension s and chars 't' and 'z'.  
function PolinomialA(M,t,z)
    s = size(M,1)
    # we need variables to work with
    variable1 = Sym(t)
    variable2 = Sym(z)
    # conjugate the variable 1, provided that this will be the variable
    # of the left polinomial
    variable1 = conjugate(variable1)
    # generate the components of the polinomial with powers of t
    poli_z = Array{SymPy.Sym}(undef, s)
    for i in 1:s
        poli_z[i] = variable2^(i-1)
    end
    # generate the components of the polinomial with powers of z
    poli_t = Array{SymPy.Sym}(undef, s)
    for i in 1:s
        poli_t[i] = (1 // i)*(variable1^i)
    end
    # multiply matrix times vector
    result = M * poli_z
    #println(result)
    # use dot product for the two vectors
    result = dot(poli_t,result)
    return result
end


"""
    elementary_differentials_csrk(M,tree)

This function calculates the CSRK elementary differential for a given 
square matrix 'M' and a given RootedTree.

"""
function elementary_differentials_csrk(M,rootedtree)
    # we'll work with the level sequence
    tree = rootedtree.level_sequence
    m = maximum(tree)
    l = length(tree)
    # create the variables called 'xi' for 1 <= i <= m
    variables = []
    for i in 1:m
        var_name = "x$i"
        var = Sym(var_name)
        push!(variables, var)
    end
    inverse_counter = l-1
    # stablish initial integrand, which is the rightmost leaf (last node of the level sequence)
    if l > 1
        integrand = integrate(PolinomialA(M,variables[tree[end]-1],variables[tree[end]]),(variables[tree[end]],0,1))
        #println(integrand)
    else
        # if the Rooted Tree is [1] or [], the elementary differential will be 1.
        return 1
    end
    while inverse_counter > 1
        # println("A_",variables[tree[inverse_counter]-1],variables[tree[inverse_counter]])
        pseudo_integrand = PolinomialA(M,variables[tree[inverse_counter]-1],variables[tree[inverse_counter]])*integrand
        integrand = integrate(pseudo_integrand,(variables[tree[inverse_counter]],0,1))
        # println(integrand)
        inverse_counter -= 1
    end
    #println(integrate(PolinomialA(M,1,variables[1])*integrand,(variables[1],0,1)))
    #multiply for the Basis_Polonimial, i.e. the Polinomial B
    #return the integral with respect to x1.
    return integrate(PolinomialA(M,1,variables[1])*integrand,(variables[1],0,1))
end



elementary_differentials_csrk

## 2x2 Matrix

Using the code for generating CSRK coefficients for a given matrix and a given RootedTree, I tried to generate and solve the equations for the matrix
\begin{equation}
M = 
\begin{bmatrix}
\alpha & \beta \\
\beta & \gamma \\
\end{bmatrix}  
\end{equation}

In [4]:
M = [Sym("a") Sym("b");
Sym("b") Sym("c")]

#elementary_differentials_csrk(M,rootedtree([1, 2]))
#resultados numericos
#a =  3.0167876
#b = -5.77444127
#c = 15.03061467
#M = [a b;
#    b c]
println(elementary_differentials_csrk(M,rootedtree([1, 2])))
println(elementary_differentials_csrk(M,rootedtree([1, 2,3])))
println(elementary_differentials_csrk(M,rootedtree([1, 2,2])))

a^2/2 + a*b + a*c/4 + b^2/2 + b*c/4 + c^2/32
a^3/4 + 3*a^2*b/4 + 13*a^2*c/72 + 109*a*b^2/144 + 53*a*b*c/144 + 13*a*c^2/288 + 37*b^3/144 + 109*b^2*c/576 + 3*b*c^2/64 + c^3/256
a^3/3 + a^2*b + a^2*c/4 + a*b^2 + a*b*c/2 + a*c^2/16 + b^3/3 + b^2*c/4 + b*c^2/16 + c^3/192


These equations are equated to $(1/2, 1/6, 1/3)$ respectively (which are the errors obtained from the difference between a bseries and its exact_solution.

Using $fsolve$ from Python, I got this solution for the variables: $(\alpha,\beta,\gamma) = (3.0167876, -5.77444127, 15.03061467)$. 

The first elementary differentials are given below:

In [5]:
a =  3.0167876
b = -5.77444127
c = 15.03061467
M = [a b;
    b c]
println(elementary_differentials_csrk(M,rootedtree([1, 2])))
println(elementary_differentials_csrk(M,rootedtree([1, 2,3])))
println(elementary_differentials_csrk(M,rootedtree([1, 2,2])))

0.499999997500000
0.166666665165591
0.333333330833333


Which are values which approximately matches with the expected values until the 8th decimal. In contrast, the coefficients given by the 4x4 matrix proposed by Miyatake are always rationals and match perfectly with the expected results for low order RootedTrees. 

In [10]:
# from page 2005
M = [-6//5 72//5 -36 24;
    72//5 -144//5 -48 72;
    -36 -48 720 -720;
    24 72 -720 720]

println(elementary_differentials_csrk(M,rootedtree([1, 2])))
println(elementary_differentials_csrk(M,rootedtree([1, 2,3])))
println(elementary_differentials_csrk(M,rootedtree([1, 2,2])))
println("Check higher order trees: ")
println(elementary_differentials_csrk(M,rootedtree([1, 2,3,4])))
println(elementary_differentials_csrk(M,rootedtree([1, 2,3,4,4,3])))
println(elementary_differentials_csrk(M,rootedtree([1, 2,2,3,3,4,5])))

1/2
1/6
1/3
Check higher order trees: 
1/24
7241/661500
3078611/617400000


Remark: the RootedTree $[1]$ is not considered provided that its output will always be 1, according to the definition of the CSRK elementary differentials.